In [1]:
from pulp import *
import random

In [2]:
class Grid:
    def __init__(self, N, blacksq=[]):
        self.N = N
        self.grid = [['-' for _ in range(N)] for _ in range(N)]
        for b in blacksq:
            if type(b) != tuple: 
                raise TypeError 
                break
            self.grid[b[0]][b[1]] = '#'
        
        # slot handling  *HACKY*
        self.positions = {'across': {}, 'down': {}}    # cell->position w/in word
        self.slots     = {'across': {}, 'down': {}}    # cell->slot e.g. (2, across)  
        self.sizes     = {'across': {}, 'down': {}}    # cell->slot size
        self.allslots  = []     # list of all slots
        cgrid = [[c for c in self.grid[i]] for i in range(len(self.grid))]
        for d in ['across','down']:
            sx = 0
            for i, row in enumerate(cgrid):
                line = ''.join(row)
                blocks = line.split('#')
                jcounter = 0
                for b in blocks:
                    if len(b) == 0:
                        jcounter += 1
                        continue
                    elif len(b) == 1:
                        jcounter += 2
                        continue
                    for bcounter in range(len(b)):
                        cell = (i,jcounter) if d=='across' else (jcounter,i)
                        self.positions[d][cell] = bcounter
                        self.slots[d][cell] = (sx,d)
                        self.sizes[d][cell] = len(b)
                        jcounter += 1
                    self.allslots.append((sx,d))
                    sx += 1
                    jcounter += 1
            cgrid = map(list, zip(*cgrid))   # transpose grid
        self.sizebyslot = {d: {self.slots[d][cell][0]: self.sizes[d][cell] for cell in self.slots[d]} \
                           for d in self.slots}
            
    def getPositionAt(self, cell, d):
        return self.positions[d][cell] if cell in self.positions[d] else None
    
    def getSlotAt(self, cell, d):
        return self.slots[d][cell] if cell in self.slots[d] else None
    
    def getSizeAt(self, cell, d):
        return self.sizes[d][cell] if cell in self.sizes[d] else None
    
    def getAllSlots(self):
        return self.allslots
    
    def getSlotSize(self, slot):
        return self.sizebyslot[slot[1]][slot[0]]
    
    def iterCells(self):
        for i in range(N):
            for j in range(N):
                if self.grid[i][j] != '#':
                    yield (i,j)
    
    def __repr__(self):
        str = ''
        for r in self.grid:
            for c in r:
                str += c + ' '
            str += '\n'
        return str

In [3]:
N = 3         # dimension of puzzle
numk = 500    # number of words

G = Grid(N, blacksq=[(0,0)])
slots = G.getAllSlots()
nums = len(slots)

print('Number slots: %d' % nums)
print(G)

# load dictionary of words as set
allwords0 = []
with open('ospd.txt', 'r') as f:
    allwords0 = [w.strip() for w in f.readlines()]
psz = set([G.getSlotSize(s) for s in G.getAllSlots()])
print('Different word lengths:', psz)
allwords = []
for size in psz:
    allwords.extend([w for w in allwords0 if len(w)==size])

allwords = random.sample(allwords, numk)

# allcosts = {w: random.randint(0,100) for w in allwords}
allcosts = [random.randint(0,100) for w in allwords]

print('Number words: %d' % len(allwords))

Number slots: 6
# - - 
- - - 
- - - 

Different word lengths: {2, 3}
Number words: 500


In [4]:
allposs = {}
for i in range(N):   # row
    for j in range(N):   # col
        c = (i,j)
        allposs[c] = {}
        pos = {d: G.getPositionAt(c,d) for d in ['across', 'down']}
        for let in 'abcdefghijklmnopqrstuvwxyz':
            allposs[c][let] = {}
            for d in ['across', 'down']:
                allposs[c][let][d] = \
                    set([(k,w) for k,w in enumerate(allwords) \
                         if  len(w) == G.getSizeAt(c, d) \
                         and w[pos[d]] == let])

In [5]:
prob = LpProblem("puzzle", LpMaximize)

poss_combos = [(w, s) for w in range(len(allwords)) for s in slots]
zvars = LpVariable.dicts('zvars', poss_combos, 0, 1, LpBinary) 

# Constr (1) : every word appears once or not at all (numk constraints)
for k in range(numk):
    prob += lpSum(zvars[(k,s)] for s in slots) <= 1
    
# Constr (2) : all slots assigned to exactly one word (nums constraints)
for s in slots:
    prob += lpSum(zvars[(k,s)] for k in range(numk)) == 1
    
# Constr (2a) : slots must contain a feasible word
for s in slots:
    ssize = G.getSlotSize(s)
    for k in range(numk):
        if len(allwords[k]) != ssize:
            prob += zvars[(k,s)] == 0
    
# Constr (3) : in every cell, zvars must match
for i in range(N):
    for j in range(N):
        c = (i,j)
        slota = G.getSlotAt(c, 'across')
        slotd = G.getSlotAt(c, 'down')
        for let in 'abcdefghijklmnopqrstuvwxyz':
            rowx = [rt[0] for rt in allposs[c][let]['across']]
            colx = [ct[0] for ct in allposs[c][let]['down']]
            
            if len(rowx)==0 or len(colx)==0:
                for k in rowx:
                    prob += zvars[(k,slota)] == 0
                for k in colx:
                    prob += zvars[(k,slotd)] == 0
                continue
                
            prob += lpSum(zvars[(k1,slota)] for k1 in rowx) == \
                    lpSum(zvars[(k2,slotd)] for k2 in colx)
                
# Objective : random costs for now
prob += lpSum(allcosts[k] * zvars[(k,s)] for k in range(numk) for s in slots)

status = prob.solve()

print(LpStatus[status])

Optimal


In [6]:
count = 0
finalw = []
for k in range(numk):
    for s in slots:
        if value(zvars[(k,s)]) > 0:
            count += 1
            finalw.append((k, s, allwords[k]))
print(count)

6


In [7]:
for l in finalw:
    print(l)

(125, (2, 'across'), 'als')
(148, (2, 'down'), 'ors')
(266, (0, 'across'), 'do')
(381, (1, 'across'), 'dor')
(402, (1, 'down'), 'dol')
(435, (0, 'down'), 'da')


In [ ]:
# # d o
# d o r
# a l s